Entrainement du modèle sur 52 fichiers en le récupérant au préalable dans le fichier de sauvegarde

Bibliothèques

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as img
import os
import cv2 as cv

#partie sklearn
import sklearn as sk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#partie tensorFLow
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop



In [2]:
size=128 #taille des images
nbDir=52 #nombre de fichiers de fleurs

Etape 1: Nettoyage des images

In [3]:
data=[]
label=[]
FlowerPath='../../Flower52'


#récupération des images et des noms d'espèces   
os.listdir(FlowerPath)
for folder in os.listdir(FlowerPath):
    for file in os.listdir(os.path.join(FlowerPath,folder)):
        if  file.endswith("jpeg"):
            label.append(folder) #on met le nom des dossiers (des espèces) dans label
            img = cv.imread(os.path.join(FlowerPath,folder, file))
            img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
            im = cv.resize(img_rgb, (size,size)) #recadre les images
            data.append(im) #on met les images dans data
                    
        else: 
            continue

data_arr = np.array(data)
label_arr = np.array(label)
y = LabelEncoder().fit_transform(label_arr)
y = to_categorical(y,nbDir)
X = data_arr/255

Etape 2: Séparation des données, 20% dans test et le reste dans train

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

Etape 3: Chargement du modèle existant

In [5]:
new_model = tf.keras.models.load_model('modele52/my_model')
new_model.summary()
new_model.compile(optimizer=Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        18464     
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 128)      0         
 2D)                                                    

C:\Users\Hugo\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [6]:
datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range = 0.20,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        vertical_flip=True)

datagen.fit(X_train)

Etape 4: Entrainement

In [7]:
batch_size=32
epochs=20

checkpoint_path = "modele52/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

history = new_model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs,
                              validation_data = (X_test,y_test),   
                              verbose = 1,
                              callbacks=[cp_callback])

C:\Users\Hugo\AppData\Local\Temp/ipykernel_15664/4150708022.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = new_model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),


Epoch 1/20
180/180 [==============================] - ETA: 0s - loss: 1.0743 - accuracy: 0.6666
Epoch 00001: saving model to modele52\cp.ckpt
180/180 [==============================] - 350s 2s/step - loss: 1.0743 - accuracy: 0.6666 - val_loss: 1.5496 - val_accuracy: 0.6455
Epoch 2/20
180/180 [==============================] - ETA: 0s - loss: 1.0349 - accuracy: 0.6880
Epoch 00002: saving model to modele52\cp.ckpt
180/180 [==============================] - 361s 2s/step - loss: 1.0349 - accuracy: 0.6880 - val_loss: 1.6639 - val_accuracy: 0.6350
Epoch 3/20
180/180 [==============================] - ETA: 0s - loss: 1.0367 - accuracy: 0.6866
Epoch 00003: saving model to modele52\cp.ckpt
180/180 [==============================] - 364s 2s/step - loss: 1.0367 - accuracy: 0.6866 - val_loss: 1.5127 - val_accuracy: 0.6406
Epoch 4/20
180/180 [==============================] - ETA: 0s - loss: 1.0426 - accuracy: 0.6823
Epoch 00004: saving model to modele52\cp.ckpt
180/180 [===========================

Etape 5: Sauvegarde

In [8]:
new_model.save('modele52/my_model')
del new_model

INFO:tensorflow:Assets written to: modele52/my_model\assets
